In [ ]:
import numpy as np
import pandas as pd

sand = pd.read_csv('231024 sand test.csv')

print(sand.shape)
sand

labels = ['Normal', "Failure"]

sand.head()

# 1st sp_data = sp[['Date', 'Pin, psi', 'Pdis, psi', 'L_sep, in', 'P_sep, psi', 'Acoustic', 'T_m_suf', 'V_in, gmrs', 'V_o, gmrs', 'V_m gmrs', 
#               'T_bearing', 'T_winding', 'label']]
sand_data = sand[['Time2', 'y','color','Head', 'Q,m3d', 'Motor temp', 'Winding_temp',  'Pin,kpa', 'Pout,kpa',  
                  'Horizon_DP','Vertical_DP', 'Vib,motor','Vib, middle', 'Vib, out',  'Acoustic' ]]
sand_data.head()


data_preprocess = sand_data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
data_preprocess

cor = data_preprocess.drop(['Time2', 'y', 'color', 'Pin,kpa', 'Pout,kpa'], axis = 1)
cor

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('default')
plt.rcParams['lines.linewidth'] = 6
plt.rcParams['figure.figsize'] = (16,12)
#plt.rcParams['font.size'] = 12


fig, ax = plt.subplots(figsize = (16,12))

mask = np.zeros_like(cor.corr(), dtype = np.bool_)
mask[np.triu_indices_from(mask)] = True
#sns.heatmap(df, cmap = 'RdYlBu_r', annot = True, mask = mask, linewidths = .5, cbar_kws= {"shrink": .5}, vmin = -1, vmax = 1)
sns.heatmap(cor.corr(), cmap = 'RdYlBu_r', annot = True, fmt = '.2f', annot_kws={'size' : 12}, square = True, 
            linewidths = 1, cbar_kws= {"shrink": .8}, vmin = -1, vmax = 1)
plt.show()

# x, y
#input_x = shifted_df.drop('y', axis=1)
# input_x = sand_data.drop(['Time', 'y'], axis = 1).values
# input_y = sand_data['y'].values

# train_normal = sand_data[:773200]
input_x = data_preprocess.drop(['Time2', 'y', 'color', 'Q,m3d', 'Pin,kpa','Pout,kpa' ,'Winding_temp'], axis = 1).values
input_y = data_preprocess['y'].values

n_features = input_x.shape[1]
input_y.shape




In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
import matplotlib.pyplot as plt

# 입력과 출력 분리
X = input_x  # 독립 변수
y = input_y  # 종속 변수

# 데이터셋 분할
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

# 정상 상태 데이터만 추출하여 스케일링
X_train_y0 = X_train[y_train == 0]
X_valid_y0 = X_valid[y_valid == 0]

def scale(X, scaler):
    return scaler.transform(X)

scaler = RobustScaler().fit(X_train_y0)

X_train_y0_scaled = scale(X_train_y0, scaler)
X_valid_scaled = scale(X_valid, scaler)
X_valid_y0_scaled = scale(X_valid_y0, scaler)
X_test_scaled = scale(X_test, scaler)

# One-Class SVM 모델
ocsvm_model = OneClassSVM(kernel='rbf', nu=0.1, gamma='auto')
ocsvm_model.fit(X_train_y0_scaled)
y_pred_ocsvm = ocsvm_model.predict(X_test_scaled)
y_pred_ocsvm = [1 if x == -1 else 0 for x in y_pred_ocsvm]  # 비정상 상태를 1로 변환

# CNN 모델
X_train_cnn = X_train_y0_scaled.reshape((X_train_y0_scaled.shape[0], X_train_y0_scaled.shape[1], 1))
X_test_cnn = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

cnn_model = Sequential([
    Conv1D(32, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    Dropout(0.2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_history = cnn_model.fit(X_train_cnn, np.zeros((X_train_cnn.shape[0], 1)), epochs=10, batch_size=32, verbose=1, validation_data=(X_test_cnn, y_test))
y_pred_cnn = (cnn_model.predict(X_test_cnn) > 0.5).astype("int32")

# 성능 평가
def evaluate_model(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return accuracy, precision, recall

# One-Class SVM의 성능 평가
accuracy_svm, precision_svm, recall_svm = evaluate_model(y_test, y_pred_ocsvm)
accuracy_cnn, precision_cnn, recall_cnn = evaluate_model(y_test, y_pred_cnn)

# 결과를 DataFrame으로 정리
results = {
    'Model': ['One-Class SVM', 'CNN', 'LSTM-AE'],
    'Accuracy': [accuracy_svm, accuracy_cnn, 90.96],
    'Precision': [precision_svm, precision_cnn, 89.90],
    'Recall': [recall_svm, recall_cnn, 89.90]
}

results_df = pd.DataFrame(results)

# HTML로 변환하여 출력
html_table = results_df.to_html(index=False)
print(html_table)

# HTML 표 시각화
from IPython.display import display, HTML
display(HTML(html_table))

# Precision-Recall 그래프 시각화
def plot_precision_recall(y_test, y_pred, model_name):
    precision, recall, _ = precision_recall_curve(y_test, y_pred)
    plt.plot(recall, precision, marker='.', label=model_name)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {model_name}')
    plt.legend()
    plt.show()

# Confusion Matrix 시각화
def plot_confusion_matrix(y_test, y_pred, model_name):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f'Confusion Matrix for {model_name}')
    plt.show()

# 모델 평가 및 시각화
plot_precision_recall(y_test, y_pred_ocsvm, 'One-Class SVM')
plot_confusion_matrix(y_test, y_pred_ocsvm, 'One-Class SVM')

plot_precision_recall(y_test, y_pred_cnn, 'CNN')
plot_confusion_matrix(y_test, y_pred_cnn, 'CNN')

# 학습 과정 시각화
def plot_history(history, model_name):
    plt.style.use('default')
    plt.rcParams['figure.figsize'] = (16,12)

    plt.plot(history.history['loss'], color="blue", linewidth=3, linestyle='solid', label='train loss')
    plt.plot(history.history['val_loss'], color="red", linewidth=3, linestyle='solid', label='valid loss')
    plt.tick_params(axis='x', labelsize=18, pad=10, length=10)
    plt.tick_params(axis='y', labelsize=18, pad=10, length=10)

    plt.xlabel('Epoch', fontsize=20, labelpad=20, fontdict={'family': 'serif', 'weight': 'bold'})
    plt.ylabel('Loss', fontsize=20, labelpad=20, fontdict={'family': 'serif', 'weight': 'bold'})
    plt.legend(loc='best', ncol=1, fontsize=18, frameon=True, shadow=True, labelspacing=.5)
    plt.title(f'{model_name} Training and Validation Loss', fontsize=22, fontdict={'family': 'serif', 'weight': 'bold'})
    plt.show()

plot_history(cnn_history, 'CNN')